## Setup

In [ ]:
Prerequisites:

In [8]:
import pandas as pd
import requests
import zipfile
from io import StringIO

Import test and training data:

In [26]:
#df_tr = pd.read_csv('kd_train.csv')
#df_test = pd.read_csv('kd_test.csv')

df_tr = pd.read_csv("https://github.com/Galeforse/DST-Assessment-01/raw/main/Data/kd_train.csv.zip")
df_test = pd.read_csv("https://github.com/Galeforse/DST-Assessment-01/raw/main/Data/kd_test.csv.zip")
df_tr.head()

,Unnamed: 0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,folds
0,44625,0,icmp,ecr_i,SF,1032,0,0,0,0,...,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.,1
1,339463,0,icmp,ecr_i,SF,1032,0,0,0,0,...,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.,1
2,321570,0,icmp,ecr_i,SF,1032,0,0,0,0,...,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.,1
3,355860,0,tcp,private,S0,0,0,0,0,0,...,0.06,0.07,0.0,0.0,1.0,1.0,0.0,0.0,neptune.,1
4,9708,0,icmp,ecr_i,SF,1032,0,0,0,0,...,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.,1


## About the Data

First we will create a data quality report on the data to get an overview of what kind of values the data contains, as well as to check for any missing data.

In [20]:
#DataFrame with columns
columns = pd.DataFrame(list(df_tr.columns.values[1:]))

#DataFrame with data types
data_types = pd.DataFrame(df_tr.dtypes, columns=['Data Type'])

#DataFrame with Count
data_count = pd.DataFrame(df_tr.count(), columns=['Count'])

#DataFrame with unique values
unique_value_counts = pd.DataFrame(columns=['Unique Values'])
for v in list(df_tr.columns.values):
    unique_value_counts.loc[v] = [df_tr[v].nunique()]

missing_data_counts = pd.DataFrame(df_tr.isnull().sum(), columns=['Missing Values'])
data_quality_report = data_types.join(data_count).join(unique_value_counts).join(missing_data_counts)
print('Data Quality Report')
data_quality_report

Data Quality Report


,Data Type,Count,Unique Values,Missing Values
Unnamed: 0,int64,444619,444619,0
duration,int64,444619,2286,0
protocol_type,object,444619,3,0
service,object,444619,65,0
flag,object,444619,11,0
src_bytes,int64,444619,3182,0
dst_bytes,int64,444619,10278,0
land,int64,444619,2,0
wrong_fragment,int64,444619,3,0
urgent,int64,444619,3,0


What we are most interested in in this project is the ability to predict normal vs non-normal traffic, therefore we are going to count the number of each type of traffic under the "label" column to get an impression of what kinds of attacks occur and how numerous they are in comparison to the normal traffic.

In [38]:
attack = df_tr.groupby("label").size().sort_values(ascending=False)
attack = pd.DataFrame(attack)
attack.reset_index(level=0, inplace=True)
attack.columns = ['Attack','Count']
attack

,Attack,Count
0,smurf.,252590
1,neptune.,96598
2,normal.,87549
3,back.,1989
4,satan.,1436
5,ipsweep.,1100
6,portsweep.,937
7,warezclient.,917
8,teardrop.,887
9,pod.,236


In [37]:
df_tr2 = df_tr.drop(["folds","is_host_login","lnum_outbound_cmds"],axis=1)
df_tr2 = df_tr2.drop(df_tr2.columns[0],axis=1)
df_tr2

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
1,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
2,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
3,0,tcp,private,S0,0,0,0,0,0,0,...,15,0.06,0.07,0.0,0.0,1.0,1.0,0.0,0.0,neptune.
4,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444614,30,udp,domain_u,SF,42,42,0,0,0,0,...,245,0.96,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal.
444615,0,tcp,private,REJ,0,0,0,0,0,0,...,8,0.03,0.07,0.0,0.0,0.0,0.0,1.0,1.0,neptune.
444616,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.
444617,0,tcp,private,S0,0,0,0,0,0,0,...,7,0.03,0.07,0.0,0.0,1.0,1.0,0.0,0.0,neptune.
